In [33]:
import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from matplotlib import style

import random
from pprint import pprint

import warnings
warnings.filterwarnings('ignore')

### Format of the data
*last coulomn of data-frome must be contain the lable and it must also be called lable

*There should be no missing values 

In [34]:
df = pd.read_csv("IRIS.csv")
df = df.rename(columns={'species':'label'})

In [35]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,label
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   label         150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


## Train-Test-Split

In [37]:
def train_test_split(df, test_size):
    
    if isinstance (test_size, float):
        test_size = round(test_size*len(df))
        
    indices = df.index.tolist()
    test_indices = random.sample(population=indices, k = test_size)

    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)

    return train_df, test_df


In [38]:
random.seed(0) #TO FREEZE THE RANDOMIZATION 
train_df, test_df = train_test_split(df, test_size=20)

# Helper function 

In [39]:
data = train_df.values
data[:5]

array([[5.1, 3.5, 1.4, 0.2, 'Iris-setosa'],
       [4.9, 3.0, 1.4, 0.2, 'Iris-setosa'],
       [4.7, 3.2, 1.3, 0.2, 'Iris-setosa'],
       [4.6, 3.1, 1.5, 0.2, 'Iris-setosa'],
       [5.0, 3.6, 1.4, 0.2, 'Iris-setosa']], dtype=object)

### Data pure?

In [40]:
def check_purity (data):
    label_column = data[:,-1]
    unique_classes = np.unique(label_column)

    if len(unique_classes)==1:
        return True
    else:
        return False

In [41]:
# check_purity(train_df[train_df.petal_width < 0.8].values)

### Classify

In [63]:
def classify_data(data):
    label_column = data[:,-1]
    unique_classes, count_unique_class = np.unique(label_column, return_counts=True)

    index = count_unique_class.argmax()
    classification = unique_classes[index]
    return classification

In [64]:
# classify_data(train_df[(train_df.petal_width > 0.8) & (train_df.petal_width < 2)].values)

In [65]:
# classify_data(train_df.values)

### Potential split 

In [100]:
# **** Pata nahi kya dikkat hai ise****

# def get_potential_split(data):

#     potential_split = {}
#     _, n_columns = data.shape
#     for column_index in range (n_columns -1):
#         potential_split[column_index] = []
#         values = data[:, column_index]
#         unique_values = np.unique(values)
        
#         for index in range (len(unique_values)):
#             if index != 0 :
#                 current_values = unique_values[index]
#                 previous_values = unique_values[index - 1]
#                 potential_split = (current_values + previous_values) / 2
                
#                 potential_split[column_index].append(potential_split)

#     return potential_split


In [105]:
# This is way to get Median of the features 

def get_potential_splits(data):
    
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):   # excluding the last column which is the label
        potential_splits[column_index] = []
        values = data[:, column_index]
        unique_values = np.unique(values)

        for index in range(len(unique_values)):
            if index != 0:
                current_value = unique_values[index]
                previous_value = unique_values[index - 1]
                potential_split = (current_value + previous_value) / 2
                
                potential_splits[column_index].append(potential_split)
    
    return potential_splits

In [106]:
# **** Plotting ****
# ptl.figure(figsize = ())
# potential_split = get_potential_splits(train_df.values)

# sns.lmplot(data= train_df, x = 'petal_width', y= 'petal_length', hue= 'label', fit_reg= False, size=6, aspect=1.5)

# # plt.vlines(x=potential_split[3], ymin=1, ymax=7)
# plt.hlines(y=potential_split[2], xmin=0, xmax=2.5)

### Split Data

In [48]:
def split_data(data, split_column, split_value):
    
    split_column_value = data[:, split_column]
# split_column_value <= split_value
    data_below = data[split_column_value <= split_value]
    data_above = data[split_column_value > split_value]
    
    return data_below, data_above

In [49]:
split_column = 3
split_value = 0.8

In [50]:
data_below, data_above = split_data(data, split_column, split_value)

In [51]:
#                           ****plotting****

# ploting_df = pd.DataFrame(data_below, columns= df.columns)
# # ploting_df = pd.DataFrame(data_above, columns= df.columns)

# sns.lmplot(data= ploting_df, x="petal_width", y= "petal_length",hue='label', fit_reg = False, size= 6, aspect = 1.5)
# plt.vlines(x=split_value, ymin=1, ymax=7)
# plt.xlim(0, 2.6)

### Lowest Overall Entropy ?

In [52]:
def calculate_entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts= True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
    
    return entropy 

In [53]:
calculate_entropy(data_above)

1.0

In [54]:
#                                                                  ******Gini Impurity******  

# def calculate_gini(data_above, data_below):
#     n_data_points = len(data_above) + len(data_below)

#     p_data_below = len(data_below) / n_data_points
#     p_data_above = len(data_above) / n_data_points

#     gini = 1 - (np.square(p_data_above)+np.square(p_data_below))
#     return gini

In [55]:
def calculate_overall_entropy(data_below, data_above):
    
    n_data_points = len(data_above) + len(data_below)

    p_data_below = len(data_below) / n_data_points
    p_data_above = len(data_above) / n_data_points

    overall_entropy = (p_data_below * calculate_entropy(data_below) + p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

In [56]:
calculate_overall_entropy(data_below, data_above)

0.6461538461538462

### Information gain ?

In [57]:
def determine_best_split(data, potential_split):
    
    overall_entropy = 1000
    
    for column_index in potential_split:
        for value in potential_split[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)
            
            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
                
    return best_split_column, best_split_value
            

In [58]:
potential_split = get_potential_splits(data)


In [59]:
determine_best_split(data, potential_split)

(3, 0.8)

# Decision Tree Algorithm 

### Representation of Tree

In [60]:
sub_tree = {"question": ["yes_answer", 
                         "no_answer"]}

In [61]:
example_tree = {"petal_width <= 0.8": ["Iris-setosa", 
                                      {"petal_width <= 1.65": [{"petal_length <= 4.9": ["Iris-versicolor", 
                                                                                        "Iris-virginica"]}, 
                                                                "Iris-virginica"]}]}

### Algorithm

In [86]:
def decision_tree_algorithm(df, counter = 0, min_samples = 2, max_depth = 5):
    
    # Data preparation 
    if counter == 0:
        global COLUMN_HEADER
        COLUMN_HEADER = df.columns
        data = df.values
    else:
        data = df
        
    # Base case
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        return classification
    
    # Recursive part
    else:
        counter +=1
        
        # Helper function 
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above  = split_data(data, split_column, split_value)
        
        # Instantiate sub-tree
        feature_name = COLUMN_HEADER[split_column]
        question = "{} <= {}".format(feature_name,split_value)
        sub_tree = {question:[]}
        
        # Find Answer (Recursion)
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth)
        
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree   

In [76]:
# tree = decision_tree_algorithm(train_df[train_df.label != "Iris-virginica"])
# tree

{'petal_width <= 0.8': ['Iris-setosa', 'Iris-versicolor']}

In [87]:
tree = decision_tree_algorithm(train_df, max_depth=3)
pprint(tree)

{'petal_width <= 0.8': ['Iris-setosa',
                        {'petal_width <= 1.65': [{'petal_length <= 4.95': ['Iris-versicolor',
                                                                           'Iris-virginica']},
                                                 'Iris-virginica']}]}
